In [15]:
import warnings
from importlib import reload
from functools import partial, lru_cache

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm, trange
from joblib import delayed
from modAL import batch
from art.metrics import empirical_robustness
from art.attacks.evasion import *
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tabulate import tabulate
from art.estimators.classification.scikitlearn import ScikitlearnSVC
from sklearn.metrics.pairwise import paired_distances, euclidean_distances
import scipy
from traceback_with_variables import printing_exc, ColorSchemes, Format

from ipynb.fs.defs import Bias
from ipynb.fs.defs.Datasets import generateData_twoPills_2D, generateData_twoPills_noNoise_2D, plot_dataset_2D

import libactive
import libadversarial
import libdatasets
from libactive import MyActiveLearner, active_split
from libadversarial import adversarial, uncertainty, random_batch
from libutil import ProgressParallel
from libdatasets import *
import librun
from librun import run

In [47]:
def bias(X, y, rand=None, config_str=None):
    dataset_str = config_str.split("__")[0]
    func = {
        "banknote": bias_banknote,
        "abalone": bias_abalone,
        "car6": bias_car,
        "cardio": bias_cardio,
        "shuttle": bias_shuttle,
        "skin": bias_skin,
    }
    return func[dataset_str](X,y)

In [48]:
def car6():
    X, y = car()
    return X[:,0:-1], y

In [56]:
matrix = {
    # Dataset fetchers should cache if possible
    # Lambda wrapper required for function to be pickleable (sent to other threads via joblib)
    "datasets": [
        #("car6", lambda: lru_cache()(car6)()),
        #("abalone", lambda: lru_cache()(abalone)()),
        ("banknote", lambda: lru_cache()(banknote)()),
    ],
    "dataset_mutators": {
        #"none": (lambda *x, **kwargs: x),
        "bias": bias,
    },
    "methods": [
        ("random", partial(random_batch, n_instances=10)),
        ("uncertainty", partial(batch.uncertainty_batch_sampling, n_instances=10)),
        ("deepfool", partial(adversarial, Attack=partial(DeepFool, verbose=False), n_instances=10)),
    ],
    "models": [
        "svm-linear"
    ],
    "meta": {
        "dataset_size": 1000,
        "labelled_size": 0.1,
        "n_runs": 10,
        "ret_classifiers": True,
    }
}

In [17]:
dataset_summary(*car())

  Instances    Classes    Features  Most common class    Least common class
-----------  ---------  ----------  -------------------  --------------------
       1728          4           7  unacc 70%            vgood 4%


* buying: vhigh, high, med, low.
* maint: vhigh, high, med, low.
* doors: 2, 3, 4, 5more.
* persons: 2, 4, more.
* lug_boot: small, med, big.
* safety: low, med, high.

In [18]:
dataset_summary(*bias_car(*car()))

  Instances    Classes    Features  Most common class    Least common class
-----------  ---------  ----------  -------------------  --------------------
       1152          4           6  unacc 55%            vgood 6%


In [57]:
#with printing_exc(fmt=Format(color_scheme=ColorSchemes.common)):
librun.run(matrix)

ValueError: X.shape[1] = 5 should be equal to 4, the number of features at training time